# NNEmbedding

# Import Modules

In [82]:
import warnings, random
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


from itertools import permutations # For making pairs

warnings.filterwarnings('ignore')

# Data loading

In [83]:
user_problem = pd.read_csv('./userId_problemId.csv', usecols=['userId', 'problemId', 'level', 'title'])
problem_category = pd.read_csv('./problem_category.csv', usecols=['problemId', 'category']) # for title-matching

In [84]:
print(user_problem.head())
print(problem_category.head())

    userId  problemId  level      title
0  sos0911       1000      1        A+B
1  sos0911       1001      1        A-B
2  sos0911       1002      7         터렛
3  sos0911       1003      8    피보나치 함수
4  sos0911       1005     13  ACM Craft
   problemId        category
0       1000      arithmetic
1       1000  implementation
2       1000            math
3       1001      arithmetic
4       1001  implementation


# Preprocessing data

In [85]:
#문제 level 10 이상만 사용 (브론즈, 실버 문제 사용 x, 골드 이상만 사용)
user_problem = user_problem[user_problem['level'] > 10]

#실제로 유저가 푼 문제의 목록화(골드 이상 문제)
unique_problem = user_problem['problemId'].unique()

#푼 문제 개수
problem_size=len(unique_problem)
print(user_problem.shape)
print("problem count:", problem_size)

#골드 이상의 문제들 빼고는 다 드랍
problem_category = problem_category[problem_category['problemId'].isin(unique_problem)]

(279500, 4)
problem count: 2755


In [87]:
#problems = selected_problems['problemId'].unique()
selected_problems = user_problem

#user별로 저장
users = dict()
users_pairs = dict()

cur_user = -1
user_prob = []
for row in selected_problems.itertuples():
    if cur_user != row.userId and cur_user != -1:
        users[cur_user] = user_prob
        user_prob = [row.problemId]
    else :
        user_prob.append(row.problemId)
    cur_user = row.userId
users[cur_user] = user_prob

#5%만 shuffle 후 사용
# for i in users.keys():
#     random.shuffle(users[i])
#     users[i] = users[i][0:len(users[i])*5//100]

# accuracy를 위한 pair 생성 키-problemId 값-[problemId로 예측하면 나와도 되는 결과problemId들]
accuracy_pair = dict()
for i in users.keys():
    for k in range(len(users[i])):
        if  users[i][k] in accuracy_pair:
            accuracy_pair[users[i][k]] = accuracy_pair[users[i][k]] + users[i][:k]+users[i][k+1:]
        else:
            accuracy_pair[users[i][k]] = users[i][:k]+users[i][k+1:]

#중복 쌍 제거
for i in accuracy_pair.keys():
    accuracy_pair[i] = np.unique(accuracy_pair[i])
    
#pair 생성
for i in users.keys():
    users_pairs[i] = torch.combinations(torch.tensor(np.unique(users[i])))

#pair쌍 합치기
problem_pairs = torch.tensor([])
for i in users_pairs.keys():
    problem_pairs = torch.cat((problem_pairs, users_pairs[i]), dim=0)

#pair의 반대 부분도 만들기
problem_pairs = torch.cat((problem_pairs, torch.flip(problem_pairs, dims=[0, 1])), dim=0)

#shuffle
random.shuffle(problem_pairs)

In [88]:
print(problem_pairs.shape)

torch.Size([80862, 2])


# Build and train neural networks

In [89]:
#training set, test set분리
train_set = problem_pairs[:len(problem_pairs)//10*9]
test_set = problem_pairs[len(problem_pairs)//10*9:]

#one hot encoding
oneHot = pd.get_dummies(problems)

#gpu 사용을 위해 cuda
torch.cuda.empty_cache()
device = "cuda" if torch.cuda.is_available() else "cpu"
cuda = torch.device(device)

In [90]:
#problem id와 one hot mapping할 dict만들기
problem_map = dict()
problem_index_to_id = dict()
index = 0
for i in sorted(problems):
    problem_map[i] = index
    problem_index_to_id[index] = i
    index += 1

In [91]:
#Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, raw_data):
        self.problems = raw_data

    def __len__(self):
        return len(self.problems)

    def __getitem__(self, idx):
        return torch.tensor(oneHot[self.problems[idx][0].item()], dtype=torch.float32, device=cuda), torch.tensor(problem_map[self.problems[idx][1].item()], device=cuda), self.problems[idx][0].item()

In [92]:
#model 생성
class EmbeddingModel(nn.Module):
    def __init__(self):
        super(EmbeddingModel, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(oneHot.shape[0], 40),
            nn.ReLU(),
            nn.Linear(40, oneHot.shape[0]),
        )
        
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [93]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y, pId) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y, pId in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            pred_pid = pred.argmax(dim=1)
            
            for i in range(len(pId)):
                #pred_pid는 index이기 때문에 다시 문제 id로 변환 해줘야 함.
                correct += int(problem_index_to_id[pred_pid[i].item()] in accuracy_pair[int(pId[i])])

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [74]:
model = EmbeddingModel().cuda()

# 파라미터 설정
learning_rate = 0.01
beta1 = 0.9
beta2 = 0.999
batch_size = 256
epochs = 5
# 옵티마이저 초기화
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(beta1, beta2))
#loss 초기화
loss_fn = nn.CrossEntropyLoss()
#기존에는 batch size = 64
train_dataloader = DataLoader(CustomDataset(train_set.to(device=cuda)), batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(CustomDataset(test_set.to(device=cuda)), batch_size=batch_size, shuffle=False)

#학습
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 7.920122  [    0/72774]
loss: 6.542366  [25600/72774]
loss: 6.455229  [51200/72774]
Test Error: 
 Accuracy: 72.7%, Avg loss: 6.447895 

Epoch 2
-------------------------------
loss: 6.155046  [    0/72774]
loss: 5.889677  [25600/72774]
loss: 6.049175  [51200/72774]
Test Error: 
 Accuracy: 88.9%, Avg loss: 6.350933 

Epoch 3
-------------------------------
loss: 4.660985  [    0/72774]
loss: 5.307161  [25600/72774]
loss: 5.758096  [51200/72774]
Test Error: 
 Accuracy: 92.9%, Avg loss: 6.356423 

Epoch 4
-------------------------------
loss: 4.234782  [    0/72774]
loss: 5.052154  [25600/72774]
loss: 5.582673  [51200/72774]
Test Error: 
 Accuracy: 93.4%, Avg loss: 6.386062 

Epoch 5
-------------------------------
loss: 4.120868  [    0/72774]
loss: 4.948657  [25600/72774]
loss: 5.479917  [51200/72774]
Test Error: 
 Accuracy: 93.8%, Avg loss: 6.421243 

Done!


# Recommend Problem

In [75]:
#category 합치기
categories = pd.DataFrame(columns = ['problemId', 'category'])
category_sum = ""
cur_problem = -1
for row in problem_category.itertuples():
    if cur_problem != row.problemId and cur_problem != -1:
        categories = categories.append({'problemId': cur_problem, 'category': category_sum}, ignore_index=True)
        category_sum = row.category
    else:
        category_sum = category_sum + "/" + str.strip(row.category)
    cur_problem = row.problemId
categories = categories.append({'problemId': row.problemId, 'category': category_sum}, ignore_index=True)
print(categories.head())
#join
total_info = user_problem.join(categories.set_index('problemId'), on='problemId')
total_info.head()

#문제 정보 매핑용
problem_info = dict()
currentProblemId = -1
for row in total_info.itertuples():
    if not row.problemId in problem_info.keys():
        problem_info[row.problemId] = (row.title, row.category, row.level)
        currentProblemId = row.problemId

  problemId                        category
0      1005  /dp/graphs/topological_sorting
1      1006                              dp
2      1007               bruteforcing/math
3      1011                            math
4      1013                    regex/string


In [76]:
#cosine 유사도를 사용하여 풀지않은 상위 몇개의 문제를 추천
def recommendProblem(user_id, user_embedding, problem_embedding, problem_list, recommend_count = 5):
    print('user {0}'.format(user_id))
    cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    res = []
    for i in problem_embedding.keys():
        if not i in problem_list:
            res.append((cos(problem_embedding[i], user_embedding[user_id]), i, problem_info[i][0], problem_info[i][1], problem_info[i][2]))
    res = sorted(res, reverse=True)[0:recommend_count]
    
    length = recommend_count if len(res) > recommend_count else len(res)
    for i in range(length):
        print('problemId: {0}, title: {1}, category: {2}, level: {3}, Similarity: {4:0.4f}'.format(res[i][1], res[i][2], res[i][3], res[i][4], res[i][0]))
    print("")
    
    return res

In [77]:
#각 영화별로 embedding 생성
problem_embedding = dict()
for param in model.parameters():
    for i in range(param.shape[1]):
        problem_embedding[problem_index_to_id[i]] = param[:,i]
    break

In [80]:
#사용자에 대한 embedding 생성
user_embedding = dict()
for i in users.keys():
    user_embedding[i] = torch.tensor(np.zeros(len(problem_embedding[1005])), device=cuda)
    for k in users[i]:
        user_embedding[i] += problem_embedding[k]
    user_embedding[i] /= len(users[i])

In [81]:
#문제 embedding과 유사도 계산 및 추천
recommend = dict()
recommend_count = 5
for i in users.keys():
    recommend[i] = recommendProblem(i, user_embedding, problem_embedding, users[i], recommend_count)

user sos0911
problemId: 11056, title: 두 부분 문자열, category: dp/string, level: 12, Similarity: 0.5527
problemId: 1240, title: 노드사이의 거리, category: bfs/dfs/graph_traversal/graphs/trees, level: 11, Similarity: 0.5485
problemId: 1028, title: 다이아몬드 광산, category: dp, level: 16, Similarity: 0.4819
problemId: 1041, title: 주사위, category: greedy/math, level: 11, Similarity: 0.4808
problemId: 15644, title: 구슬 탈출 3, category: bfs/graph_traversal/graphs/implementation/simulation, level: 15, Similarity: 0.4791

user dnfl182
problemId: 2931, title: 가스관, category: implementation/simulation, level: 13, Similarity: 0.5520
problemId: 3745, title: 오름세, category: binary_search/lis, level: 14, Similarity: 0.5491
problemId: 16639, title: 괄호 추가하기 3, category: dp, level: 15, Similarity: 0.5441
problemId: 1795, title: 마알, category: bfs/graph_traversal/graphs/implementation, level: 13, Similarity: 0.5252
problemId: 3747, title: 완벽한 선거!, category: 2_sat/graphs/scc, level: 17, Similarity: 0.4987

user hja5432
problem

problemId: 5896, title: 효율적으로 소 사기, category: data_structures/greedy/priority_queue, level: 17, Similarity: 0.5356
problemId: 1728, title: 구슬 굴리기, category: binary_search/bruteforcing/math, level: 16, Similarity: 0.5205
problemId: 5624, title: 좋은 수, category: dp, level: 13, Similarity: 0.4585
problemId: 3066, title: 브리징 시그널, category: binary_search/lis, level: 14, Similarity: 0.4574
problemId: 10747, title: Censoring, category: hashing/kmp/string, level: 18, Similarity: 0.4186

user dozzing729
problemId: 19566, title: 수열의 구간 평균, category: data_structures/prefix_sum/tree_set, level: 14, Similarity: 0.5249
problemId: 4386, title: 별자리 만들기, category: graphs/mst, level: 12, Similarity: 0.5150
problemId: 19644, title: 좀비 떼가 기관총 진지에도 오다니, category: data_structures/greedy/prefix_sum/queue, level: 13, Similarity: 0.5000
problemId: 17178, title: 줄서기, category: data_structures/implementation/simulation/stack, level: 11, Similarity: 0.4655
problemId: 18407, title: 가로 블록 쌓기, category: coordinate_co

problemId: 15938, title: 더위 피하기, category: dp, level: 13, Similarity: 0.5134
problemId: 2463, title: 비용, category: data_structures/disjoint_set, level: 16, Similarity: 0.5079
problemId: 20007, title: 떡 돌리기, category: dijkstra/graphs, level: 12, Similarity: 0.4811
problemId: 4256, title: 트리, category: divide_and_conquer/recursion/trees, level: 13, Similarity: 0.4772
problemId: 13144, title: List of Unique Numbers, category: two_pointer, level: 12, Similarity: 0.4749

user potion
problemId: 14897, title: 서로 다른 수와 쿼리 1, category: coordinate_compression/data_structures/mo/offline_queries/segtree, level: 19, Similarity: 0.5740
problemId: 1662, title: 압축, category: data_structures/recursion/stack, level: 11, Similarity: 0.5375
problemId: 11402, title: 이항 계수 4, category: combinatorics/dp/lucas/math/number_theory, level: 16, Similarity: 0.5195
problemId: 13418, title: 학교 탐방하기, category: graphs/mst, level: 13, Similarity: 0.5063
problemId: 12438, title: 새로운 달력 (Large), category: math/number_the

KeyboardInterrupt: 

11056번 문제는 sos0911 사용자가 풀어보지 않은 문제